In [1]:
!pip install -q together
!pip install -q FlagEmbedding
!pip install -q peft
!pip install -q faiss-gpu

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
from FlagEmbedding import FlagModel
from FlagEmbedding import FlagReranker

import os
import torch
import faiss
import re
import json
import numpy as np
import requests

In [ ]:
# encode_queries()和encode()的区别：
# for s2p (short query to long passage) retrieval task, suggest to use encode_queries() which will automatically add the instruction to each query
# corpus in retrieval task can still use encode() or encode_corpus(), since they don't need instruction

model = FlagModel('BAAI/bge-base-en-v1.5', query_instruction_for_retrieval="Represent this sentence for searching relevant passages:", use_fp16=True)
# print(model.device)

reranker = FlagReranker('BAAI/bge-reranker-base', use_fp16=True, device='cuda')
# print(reranker.device)

In [4]:
# query = "Tell me the exam information for the database class."
query = "what is happening in Los Angeles?"

courses_numbers = ['544', '566', '585', '596', '599', '626', '677', '699']
course_names = ['Applied Natural Language Processing (NLP)', 'Deep Learning and its Applications (DL)', 'Database Systems (database)', 'Scientific Computing and Visualization',
         'Distributed Systems', 'Text as Data', 'Advanced Computer Vision (CV)', 'Robotic Perception (Robotics)']

full_course_info = dict(zip(['CSCI' + num for num in courses_numbers], ['CSCI' + num + ' ' + name for num, name in zip(courses_numbers, course_names)]))

name_to_num = dict(zip(course_names, courses_numbers))

In [6]:
full_course_info

{'CSCI544': 'CSCI544 Applied Natural Language Processing (NLP)',
 'CSCI566': 'CSCI566 Deep Learning and its Applications (DL)',
 'CSCI585': 'CSCI585 Database Systems (database)',
 'CSCI596': 'CSCI596 Scientific Computing and Visualization',
 'CSCI599': 'CSCI599 Distributed Systems',
 'CSCI626': 'CSCI626 Text as Data',
 'CSCI677': 'CSCI677 Advanced Computer Vision (CV)',
 'CSCI699': 'CSCI699 Robotic Perception (Robotics)'}

In [ ]:
def build_vector_database(course_names):
    course_embeddings = model.encode(course_names)
    np_course_embeddings = np.array(course_embeddings).astype('float32')

    # with normalization -> bacome cosine similarity search
    faiss.normalize_L2(np_course_embeddings)

    # IndexFlatIP: dot-product search, IndexFlatL2: L2 search
    index_innerproduct = faiss.IndexFlatIP(len(course_embeddings[0]))
    res = faiss.StandardGpuResources()  # Create GPU resources
    gpu_index = faiss.index_cpu_to_gpu(res, 0, index_innerproduct) # Move index to GPU

    gpu_index.add(np_course_embeddings)

    return gpu_index

database = build_vector_database(course_names)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
def search_(query, database, embed_dim, topk=1):
    query_embed = model.encode_queries(query)
    query_embed = np.array(query_embed).astype('float32')

    _, idx = database.search(query_embed.reshape((1, embed_dim)), topk) # (768,) -> (1, 768)
    idx = idx.reshape(-1)

    ret = [course_names[i] for i in idx]

    return ret

def find_syllabus(query, courses_numbers, course_names, name_to_num, model, database):
    def find_course():
        for num in courses_numbers:
            if num in query:
                return num

        result = search_(query, database, embed_dim=database.d, topk=1)
        return result[0]

    result = find_course()

    if result in courses_numbers:
        return 'CSCI' + result + '.txt'
    else:
        return 'CSCI' + name_to_num[result] + '.txt'

In [ ]:
print(find_syllabus(query, courses_numbers, course_names, name_to_num, model, database))

In [ ]:
# 读取txt文件并按"\n\n"切割 -> 按照需求自行修改！！！
def read_and_split_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    segments = content.split('##')
    # print(segments)

    return [seg.strip() for seg in segments]  # 变成了一个list

# 计算query和每个文档分段的相似度，并返回排序后的结果
def compute_similarity(query, segments):
    # encode_queries是专门用于encode query的
    query_embedding = model.encode_queries(query)  # [512, ]
    segment_embeddings = model.encode(segments)  # [num_seg, 512]

    # 官方提供的。只计算点积，不计算cos-simi
    similarities = [query_embedding @ segment_embedding.T for segment_embedding in segment_embeddings]

    # 将分数、query和文档分段组合并排序 -> 降序
    results = sorted(zip(similarities, segments), key=lambda x: x[0], reverse=True)
    # print(results)

    return results

# 定义生成查询和文档组合的函数
def generate_query_passage_pairs(query, passages):
    return [[query[0], passage] for passage in passages]

In [5]:
def RAG(query):

    if query is None:
        raise ValueError('Your query cannot be empty!')

    k = 3

    final_syllabus = find_syllabus(query, courses_numbers, course_names, name_to_num, model, database)
    print(final_syllabus)

    # file_path = '/content' + os.sep + final_syllabus
    file_path = '/content/drive/MyDrive/CSCI544/Project/syllabus/' + final_syllabus
    knowledge_base = read_and_split_file(file_path)

    query = [query]
    all_similarity = compute_similarity(query, knowledge_base)

    results = []
    for score, segment in all_similarity:
        if score > 0.25:
            results.append(segment)

    query_passage_pairs = generate_query_passage_pairs(query, results)

    if query_passage_pairs:
        scores = reranker.compute_score(query_passage_pairs)
        sorted_query_passage_pairs = sorted(zip(scores, query_passage_pairs), key=lambda x: x[0], reverse=True)

        sorted_results = [(score, pair[1]) for score, pair in sorted_query_passage_pairs]

        final_results = []
        for score, passage in sorted_results:
            # if score > 0:
            final_results.append((score, passage))

        final_results = final_results[:k]
    else:
        final_results = [(0.0, 'There is no relevant information for the given query!')]

    all_segments = '\n\n'.join([result[1] for result in final_results])
    class_found = final_syllabus[:-len('.txt')]

    return f'Here are class information for {full_course_info[class_found]}:\n{all_segments}'

In [103]:
import requests

def general_news_report(query):
    """
    Fetches news articles based on the user's query.

    Args:
        query (str): The search query for the news topics.
        max_articles (int): Maximum number of articles to retrieve.

    Returns:
        str: A formatted string containing the titles and descriptions of the retrieved articles.
    """
    # Retrieve the News API key from user data or configuration
    new_api_key = userdata.get('new_api')  # Ensure this variable is correctly set in your environment
    max_articles=5
    # News API endpoint
    endpoint = 'https://newsapi.org/v2/everything'

    # Set query parameters
    params = {
        'q': query,                # Use the user's query
        'language': 'en',          # Language preference
        'sortBy': 'publishedAt',   # Order by most recent
        'apiKey': new_api_key,
    }

    # Send request to News API
    response = requests.get(endpoint, params=params)

    # Check for successful response
    if response.status_code == 200:
        news_data = response.json()
        articles = news_data.get('articles', [])[:max_articles]  # Limit the number of articles
        final_res = ""

        # Parse and append each article's title and description
        for article in articles:
            title = article.get('title', 'No Title')
            description = article.get('description', 'No Description')
            final_res += f"Title: {title}\n"
            final_res += f"Description: {description}\n\n"

        # Return the formatted string
        return final_res.strip()

    else:
        # Handle errors and return the response status
        error_message = f"Failed to retrieve news: {response.status_code} - {response.reason}"
        print(error_message)
        return error_message

In [ ]:
print(general_news_report("Los Angeles"))

In [62]:
import json
import requests

def fetch_weather(params) -> str:
    """
    Fetch weather information based on the provided city name.

    Args:
        params (str): A JSON-formatted string containing the city name and key selections.

    Returns:
        str: A stringified dictionary containing the selected weather details.
    """
    # print("Step 1: Parsing input JSON...")
    # Parse the input JSON string to extract parameters
    city_name = params

    # Extract city name and key selections
    # print("Step 2: Extracting city name and key selection...")
    key_selection = {
            "FeelsLikeC": "FeelsLikeC",  # Feels like temperature in Celsius
            "temp_C": "temp_C",         # Current temperature in Celsius
            'weatherDesc': 'weatherDesc',
            "humidity": "humidity"      # Humidity percentage
        }

    if not city_name:
        # print("Error: 'city_name' is missing.")
        return "Error: 'city_name' is required in the input parameters."

    # print(f"City name: {city_name}")
    # print(f"Key selection: {key_selection}")

    # Make the API call to wttr.in
    # print("Step 3: Making the API call...")
    try:
        response = requests.get(f"https://wttr.in/{city_name}?format=j1", timeout=10)
        # print("Step 4: API call complete. Status code:", response.status_code)
        response.raise_for_status()
        weather_data = response.json()
        # print("Step 5: API response JSON parsed successfully.")
    except requests.RequestException as e:
        # print("Error: API call failed.")
        return f"Error fetching weather data: {e}"

    # Extract the required weather information based on the key selection
    # print("Step 6: Extracting weather information from API response...")
    try:
        current_condition = weather_data.get("current_condition", [{}])[0]
        # print("Current condition data:", current_condition)
        result = {}

        for display_key, json_key in key_selection.items():
            # Extract the value from 'current_condition'
            # print(f"Extracting {display_key} using key '{json_key}'...")
            result[display_key] = current_condition.get(json_key, "N/A")

        # print("Step 7: Weather information extraction complete.")
    except KeyError as e:
        # print("Error: Key missing in the API response.")
        return f"Error processing weather data: Key {e} not found in the response."

    # Return the result as a string
    # print("Step 8: Returning the result.")
    return str(result)

In [9]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "RAG",
            "description": "Retrieve the relevant section in the given knowledge base when the user asks information about courses or syllabus.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The original question that the user asks exactly, no need to rephrase.",
                    },
                },
                "required": ["query"],
                "additionalProperties": False,
            },
        }
    },
     {
            "type": "function",
            "function": {
                "name": "fetch_weather",
                "description": "Fetches the current weather for a specified city with user-defined key selections.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "params": {
                            "type": "string",
                            "description": (
                                "A string containing: "
                                "the name of the city (e.g., 'London')"
                            ),
                        },
                    },
                    "required": ["params"],
                    "additionalProperties": False,
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "general_news_report",
                "description": "Fetches recent news articles based on the user's query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The user's search query for the news (e.g., 'artificial intelligence').",
                        },
                    },
                    "required": ["query"],
                    "additionalProperties": False,
                },
            },
        },
]

In [86]:
toolPrompt = f"""
You have access to the following functions:

Use the function '{tools_list[0]['function']['name']}' to '{tools_list[0]['function']['description']}'.
The parameters are: {json.dumps(tools_list[0]['function']['parameters']['properties'])}, where {tools_list[0]['function']['parameters']['required']} are required.

Use the function '{tools_list[1]['function']['name']}' to '{tools_list[1]['function']['description']}':
The parameters are: {json.dumps(tools_list[1]['function']['parameters']['properties'])}, where {tools_list[1]['function']['parameters']['required']} are required.

Use the function '{tools_list[2]['function']['name']}' to '{tools_list[2]['function']['description']}':
The parameters are: {json.dumps(tools_list[2]['function']['parameters']['properties'])}, where {tools_list[2]['function']['parameters']['required']} are required.

If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=example_function_name>{{\"example_name\": \"example_value\"}}</function>

Reminder:
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- If there is no function call available, return nothing!

"""

messages = [
  	{
        "role": "system",
        "content": toolPrompt,
    },
    {
        "role": "user",
        "content": query,
    },

]

print(messages)

[{'role': 'system', 'content': '\nYou have access to the following functions:\n\nUse the function \'RAG\' to \'Retrieve the relevant section in the given knowledge base when the user asks information about courses or syllabus.\'.\nThe parameters are: {"query": {"type": "string", "description": "The original question that the user asks exactly, no need to rephrase."}}, where [\'query\'] are required.\n\nUse the function \'fetch_weather\' to \'Fetches the current weather for a specified city with user-defined key selections.\':\nThe parameters are: {"params": {"type": "string", "description": "A string containing: the name of the city (e.g., \'London\')"}}, where [\'params\'] are required.\n\nUse the function \'general_news_report\' to \'Fetches recent news articles based on the user\'s query.\':\nThe parameters are: {"query": {"type": "string", "description": "The user\'s search query for the news (e.g., \'artificial intelligence\')."}}, where [\'query\'] are required.\n\nIf you choose 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "NousResearch/Hermes-2-Pro-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")

In [88]:
def generate_tool_answer(input):
    inputs = tokenizer.apply_chat_template(input, tools=tools_list, return_dict=True, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    out = model.generate(**inputs, max_new_tokens=1024, pad_token_id=tokenizer.eos_token_id)
    res = tokenizer.decode(out[0][len(inputs["input_ids"][0]):])
    return res

res = generate_tool_answer(messages)
print(res)

<tool_call>
{"name": "general_news_report", "arguments": {"query": "Los Angeles news"}}    
</function><|im_end|>


In [89]:
import json
import re

def parse_tool_call(response_str: str):
    pattern = r">(.*?)</"
    match = re.search(pattern, response_str, flags=re.DOTALL)
    if not match:
        return None

    tool_call_str = match.group(1).strip()

    try:
        tool_call_data = json.loads(tool_call_str)
    except json.JSONDecodeError:
        return None

    function_name = tool_call_data.get("name")
    arguments = tool_call_data.get("arguments", {})

    return {
        "function": function_name,
        "arguments": arguments
    }



parsed_response = parse_tool_call(res)
print(parsed_response["function"])
print(parsed_response["arguments"])

general_news_report
{'query': 'Los Angeles news'}


In [90]:
import transformers
import torch

def generate_answer(input):
    inputs = tokenizer.apply_chat_template(input, return_dict=True, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    out = model.generate(**inputs, max_new_tokens=1024, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(out[0][len(inputs["input_ids"][0]):])

In [91]:
def parse_answer(answer: str):
    cleaned = re.sub(r"\s*## Answer:\s*", "", answer)
    cleaned = cleaned.replace("<|im_end|>", "")
    return cleaned.strip()

In [59]:
if parsed_response:
    available_functions = {
        "RAG": RAG,
        "fetch_weather": fetch_weather,
        "general_news_report": general_news_report,
    }

    if parsed_response["function"] not in available_functions:
        available_function_names = "\n".join(available_functions.keys())
        raise NotImplementedError(
            f"Function {parsed_response['function']} is not implemented. "
            f"Our available functions are:\n\n{available_function_names}"
        )

    arguments = parsed_response["arguments"]
    function_to_call = available_functions[parsed_response["function"]]
    # print(function_to_call)
    # print(arguments)
    # print()

    result = function_to_call(**arguments)
    # print("function call result:", result)
    # print()

    answer_prompt = "You are a very helpful assistant. Please answer user's question according to given information. Trust the given information, it is completely align with the user's question."

    new_messages = [
        {
            "role": "system",
            "content": answer_prompt,
        },
        {
            "role": "user",
            "content": f"""
                        ## Question:
                        {query}

                        ## Information:
                        {result}
                        """
        },
    ]

    # print(new_messages)
    # print("-----------------------")

    res = generate_answer(new_messages)
    parsed_res = parse_answer(res)
    print("Answer from the LLM: \n", parsed_res)
else:
    print("No function call found in the response")

Answer from the LLM: 
 In Los Angeles, several events are happening:

1. Mercedes-Benz is developing a "solar paint" technology that could revolutionize electric vehicle charging. This innovative solar module is a thin, wafer-like coating that can be applied to the bodywork of electric cars, allowing them to harness solar energy more efficiently.

2. The Kansas City Chiefs have won the AFC West for the 9th straight season, thanks to a game-winning field goal by Matthew Wright against the Los Angeles Chargers.

3. Los Angeles Rams wide receiver Puka Nacua showcased remarkable skills with 178 scrimmage yards, two touchdowns, and two incredible catches in their victory over the Buffalo Bills.

These events demonstrate the diverse range of activities and innovations taking place in Los Angeles.


In [100]:
def main():
    previous_RAG_info = None  # Placeholder for RAG information from prior turns
    conversation_count = 0
    chat_history = []  # To store the latest 10 conversations

    while True:
        # Get user query
        user_query = input("User: ")
        if user_query.lower() in ["exit", "quit"]:
            print("Exiting conversation.")
            break
        use_previous_RAG = False
        # Decide if previous RAG info should be used (dummy condition for now)
        if conversation_count > 0 and previous_RAG_info:
            use_previous_RAG = False  # Replace with actual condition
        else:
            use_previous_RAG = False

        if use_previous_RAG:
            user_query = f"{user_query}\n\nAdditional context:\n{previous_RAG_info}"

        # Add the new user query to the chat history
        chat_history.append({"role": "user", "content": user_query})

        # Ensure the chat history is limited to the latest 10 exchanges
        if len(chat_history) > 20:  # Each "exchange" is a user+assistant pair
            chat_history = chat_history[-20:]

        # Construct messages with chat history
        messages = [{"role": "system", "content": toolPrompt}] + chat_history

        # Call the LLM to decide if tool usage needed
        tool_response = generate_tool_answer(messages)
        parsed_tool_response = parse_tool_call(tool_response)
        # print("Tool:", parsed_tool_response)

        # Add assistant's response to chat history
        if parsed_tool_response:
            available_functions = {
                "RAG": RAG,
                "fetch_weather": fetch_weather,
                "general_news_report": general_news_report,
            }

            function_name = parsed_tool_response["function"]
            arguments = parsed_tool_response["arguments"]

            if function_name in available_functions:
                result = available_functions[function_name](**arguments)
                if function_name == "RAG":
                    previous_RAG_info = result  # Store RAG info for future turns
            else:
                print(f"Unknown function: {function_name}")

            # Prepare the answer prompt for the second LLM call
            answer_prompt = (
                "You are a very helpful assistant. Please answer the user's question using the information provided below. "
                "Trust the given information, as it is aligned with the user's query.\n\n"
                f"## Question:\n{user_query}\n\n"
                f"## Information:\n{result}"
            )

            # Create new LLM messages
            new_messages = [{"role": "system", "content": "You are a helpful assistant."}]
            new_messages += chat_history
            new_messages.append({"role": "user", "content": answer_prompt})

            # Call the LLM again with the tool's result
            response = generate_answer(new_messages)
            assistant_response = parse_answer(response)

            print("Agent:", assistant_response)
            chat_history.append({"role": "assistant", "content": assistant_response})
        else:
            messages = [{"role": "system", "content": "You are a helpful assistant."}]
            messages += chat_history
            messages.append({"role": "user", "content": user_query})

            # Call the LLM to generate a response
            response = generate_answer(messages)
            assistant_response = parse_answer(response)
            print("Agent:", assistant_response)
            chat_history.append({"role": "assistant", "content": assistant_response})

        print()
        print("**********split**********")
        print()

        # Increment conversation count
        conversation_count += 1


In [102]:
main()

User: please remember this number 123
Tool: None
Agent: I will remember it. Is there anything else you would like me to remember or help you with?

**********split**********

User: what number did I type?
Tool: None
Agent: I'm happy to assist! You mentioned the number 123, is there anything else I can help you with? If you have a new question or need further help, please feel free to ask.

**********split**********

User: what is happening in Los Angeles?
Tool: {'function': 'general_news_report', 'arguments': {'query': 'Los Angeles'}}
Agent: Currently, several noteworthy events are happening in Los Angeles. Mercedes-Benz is researching a new type of solar module that could revolutionize electric vehicle charging. Additionally, the Kansas City Chiefs have won the AFC West for the 9th straight season. In other sports news, Rams WR Puka Nacua had an impressive performance, leading to the Tom Brady LFG Player of the Game. Lastly, the Mets have become the new kings of New York by signing Ju